<a href="https://colab.research.google.com/github/Eddycrack864/UVR5-NO-UI/blob/main/UVR5_NO_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🎵 UVR5 NO UI 🎵**

Colab created by **[Not Eddy (Spanish Mod)](http://discord.com/users/274566299349155851)** in **[AI HUB](https://discord.gg/aihub)** community.

<br>**This is a CLI type colab created based on the [beveradb's python-audio-separator](https://github.com/karaokenerds/python-audio-separator) repository.<br>
If you liked this colab you can star my repo on [GitHub](https://github.com/Eddycrack864/UVR5-NO-UI)**.<br>

<br>You can donate to the original UVR5 project here:<br>
[!["Buy Me A Coffee"](https://www.buymeacoffee.com/assets/img/custom_images/orange_img.png)](https://www.buymeacoffee.com/uvr5)

<br>**How To Use: https://github.com/Eddycrack864/UVR5-NO-UI#how-to-use**

![](https://count.nett.moe/get/uvr5_no_ui_colab/img?theme=rule34)

In [ ]:
#@title # **Installation**
#@markdown ####This takes around 2 minutes
from IPython.display import clear_output
import subprocess
import os

colab_path = "/content"
kaggle_path = "/kaggle/working"

if os.path.exists(colab_path):
  print("Welcome to Colab Notebook")
  from google.colab import drive
  drive.mount('/content/drive')
  path = "/content"
elif os.path.exists(kaggle_path):
  print("Welcome to Kaggle Notebook")
  path = "/kaggle/working"

!pip install "audio-separator[gpu]==0.17.5"
subprocess.run(["pip", "install", "demucs"])
!pip install aria2
!pip install yt_dlp
!mkdir models
!mkdir temp
!aria2c https://huggingface.co/Eddycrack864/Drumsep/resolve/main/modelo_final.th -o models/drumsep.th
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/

clear_output()
print('Installation done !')

# <small> **Select Separation method:**

### ***BS-Roformer***

In [ ]:
#@markdown #**Separation! BS-Roformer and Mel Band Roformer Only**
import os
import glob
import subprocess
import yt_dlp

def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(f'{path}/temp', '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(audio_input, output_folder, extensions, output_format, model):
    found_files = []

    dictmodel = {
        'BS-Roformer-Viperx-1297.ckpt': 'model_bs_roformer_ep_317_sdr_12.9755.ckpt',
        'BS-Roformer-Viperx-1296.ckpt': 'model_bs_roformer_ep_368_sdr_12.9628.ckpt',
        'BS-Roformer-Viperx-1053.ckpt': 'model_bs_roformer_ep_937_sdr_10.5309.ckpt',
        'Mel-Roformer-Viperx-1143.ckpt': 'model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt'
    }
    roformer_model = dictmodel[model]

    if checker(audio_input):
        downloader(audio_input)
        audio_input = f"{path}/temp"

    for audio_files in os.listdir(audio_input):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(audio_input, audio_files)
            prompt = ["audio-separator", file_path, "-m", f"{roformer_model}", f"--output_dir={output_folder}", f"--output_format={output_format}", "--normalization=0.9", f"--mdxc_overlap={overlap}", f"--mdxc_segment_size={segment_size}"]
            print(f"Processing file: {audio_files}")
            subprocess.run(prompt)
            print(f"File: {audio_files} processed!")

    if audio_input == f"{path}/temp":
        temp_files = glob.glob(f"{path}/temp/*")
        for file in temp_files:
            os.remove(file)

#@markdown Input path for audio files or link:
audio_input = "Place the path or link here" #@param {type:"string"}
#@markdown You can paste the link to the video/audio from many sites, check the complete list [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)

#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "BS-Roformer-Viperx-1297.ckpt" #@param ["BS-Roformer-Viperx-1297.ckpt", "BS-Roformer-Viperx-1296.ckpt", "BS-Roformer-Viperx-1053.ckpt", "Mel-Roformer-Viperx-1143.ckpt"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Amount of overlap between prediction windows.
overlap = 4 #@param {type:"slider", min:2, max:4, step:1}
#@markdown Larger consumes more resources, but may give better results.
segment_size = 256 #@param {type:"slider", min:32, max:4000, step:32}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(audio_input, output_folder, extensions, output_format, model)

### ***MDX23C***

In [ ]:
#@markdown #**Separation! MDX23C Only**
import os
import glob
import subprocess
import yt_dlp

def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(f'{path}/temp', '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(audio_input, output_folder, extensions, output_format, model, segment_size, overlap):
    found_files = []

    if checker(audio_input):
        downloader(audio_input)
        audio_input = f"{path}/temp"

    for audio_files in os.listdir(audio_input):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(audio_input, audio_files)
            prompt = ["audio-separator", file_path, "-m", f"{model}", f"--output_dir={output_folder}", f"--output_format={output_format}", "--normalization=0.9", f"--mdxc_segment_size={segment_size}", f"--mdxc_overlap={overlap}"]
            if denoise:
              prompt.append("--mdx_enable_denoise")
            print(f"Processing file: {audio_files}")
            subprocess.run(prompt)
            print(f"File: {audio_files} processed!")

    if audio_input == f"{path}/temp":
        temp_files = glob.glob(f"{path}/temp/*")
        for file in temp_files:
            os.remove(file)

#@markdown Input path for audio files or link:
audio_input = "Place the path or link here" #@param {type:"string"}
#@markdown You can paste the link to the video/audio from many sites, check the complete list [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)

#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "MDX23C-8KFFT-InstVoc_HQ_2.ckpt" #@param ["MDX23C_D1581.ckpt", "MDX23C-8KFFT-InstVoc_HQ.ckpt", "MDX23C-8KFFT-InstVoc_HQ_2.ckpt"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Larger consumes more resources, but may give better results.
segment_size = 256 #@param {type:"slider", min:32, max:4000, step:32}
#@markdown Amount of overlap between prediction windows.
overlap = 8 #@param {type:"slider", min:2, max:50, step:1}
#@markdown Enable denoising during separation.
denoise = True #@param {type:"boolean"}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(audio_input, output_folder, extensions, output_format, model, segment_size, overlap)

### ***MDX-NET***

In [ ]:
#@markdown #**Separation! MDX-NET Only**
import os
import glob
import subprocess
import yt_dlp

def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(f'{path}/temp', '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(audio_input, output_folder, extensions, output_format, model, segment_size, overlap, denoise):
    found_files = []

    if checker(audio_input):
        downloader(audio_input)
        audio_input = f"{path}/temp"

    for audio_files in os.listdir(audio_input):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(audio_input, audio_files)
            prompt = ["audio-separator", file_path, "-m", f"{model}", f"--output_dir={output_folder}", f"--output_format={output_format}", "--normalization=0.9", f"--mdx_segment_size={segment_size}", f"--mdx_overlap={overlap}"]
            if denoise:
                prompt.append("--mdx_enable_denoise")
            print(f"Processing file: {audio_files}")
            subprocess.run(prompt)
            print(f"File: {audio_files} processed!")

    if audio_input == f"{path}/temp":
        temp_files = glob.glob(f"{path}/temp/*")
        for file in temp_files:
            os.remove(file)

#@markdown Input path for audio files or link:
audio_input = "Place the path or link here" #@param {type:"string"}
#@markdown You can paste the link to the video/audio from many sites, check the complete list [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)

#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "UVR-MDX-NET-Inst_HQ_4.onnx" #@param ["UVR-MDX-NET-Inst_full_292.onnx", "UVR-MDX-NET_Inst_187_beta.onnx", "UVR-MDX-NET_Inst_82_beta.onnx", "UVR-MDX-NET_Inst_90_beta.onnx", "UVR-MDX-NET_Main_340.onnx", "UVR-MDX-NET_Main_390.onnx", "UVR-MDX-NET_Main_406.onnx", "UVR-MDX-NET_Main_427.onnx", "UVR-MDX-NET_Main_438.onnx", "UVR-MDX-NET-Inst_HQ_1.onnx", "UVR-MDX-NET-Inst_HQ_2.onnx", "UVR-MDX-NET-Inst_HQ_3.onnx", "UVR-MDX-NET-Inst_HQ_4.onnx", "UVR_MDXNET_Main.onnx", "UVR-MDX-NET-Inst_Main.onnx", "UVR_MDXNET_1_9703.onnx", "UVR_MDXNET_2_9682.onnx", "UVR_MDXNET_3_9662.onnx", "UVR-MDX-NET-Inst_1.onnx", "UVR-MDX-NET-Inst_2.onnx", "UVR-MDX-NET-Inst_3.onnx", "UVR_MDXNET_KARA.onnx", "UVR_MDXNET_KARA_2.onnx", "UVR_MDXNET_9482.onnx", "UVR-MDX-NET-Voc_FT.onnx", "Kim_Vocal_1.onnx", "Kim_Vocal_2.onnx", "Kim_Inst.onnx", "Reverb_HQ_By_FoxJoy.onnx", "UVR-MDX-NET_Crowd_HQ_1.onnx", "kuielab_a_vocals.onnx", "kuielab_a_other.onnx", "kuielab_a_bass.onnx", "kuielab_a_drums.onnx", "kuielab_b_vocals.onnx", "kuielab_b_other.onnx", "kuielab_b_bass.onnx", "kuielab_b_drums.onnx"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Larger consumes more resources, but may give better results.
segment_size = 256 #@param {type:"slider", min:32, max:4000, step:32}
#@markdown Amount of overlap between prediction windows.
overlap = "0.25" #@param ["0.25", "0.5", "0.75", "0.99"]
#@markdown Enable denoising during separation.
denoise = True #@param {type:"boolean"}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(audio_input, output_folder, extensions, output_format, model, segment_size, overlap, denoise)

### ***VR ARCH***

In [ ]:
#@markdown #**Separation! VR ARCH Only**
import os
import glob
import subprocess
import yt_dlp

def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(f'{path}/temp', '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(audio_input, output_folder, extensions, output_format, model, window_size, aggression, tta, high_end_process):
    found_files = []

    if checker(audio_input):
        downloader(audio_input)
        audio_input = f"{path}/temp"

    for audio_files in os.listdir(audio_input):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(audio_input, audio_files)
            prompt = ["audio-separator", file_path, "-m", f"{model}", f"--output_dir={output_folder}", f"--output_format={output_format}", "--normalization=0.9", f"--vr_window_size={window_size}", f"--vr_aggression={aggression}"]
            if tta:
                prompt.append("--vr_enable_tta")
            if high_end_process:
              prompt.append("--vr_high_end_process")
            print(f"Processing file: {audio_files}")
            subprocess.run(prompt)
            print(f"File: {audio_files} processed!")

    if audio_input == f"{path}/temp":
        temp_files = glob.glob(f"{path}/temp/*")
        for file in temp_files:
            os.remove(file)

#@markdown Input path for audio files or link:
audio_input = "Place the path or link here" #@param {type:"string"}
#@markdown You can paste the link to the video/audio from many sites, check the complete list [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)

#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "UVR-DeEcho-DeReverb.pth" #@param ["1_HP-UVR.pth", "2_HP-UVR.pth", "3_HP-Vocal-UVR.pth", "4_HP-Vocal-UVR.pth", "5_HP-Karaoke-UVR.pth", "6_HP-Karaoke-UVR.pth", "7_HP2-UVR.pth", "8_HP2-UVR.pth", "9_HP2-UVR.pth", "10_SP-UVR-2B-32000-1.pth", "11_SP-UVR-2B-32000-2.pth", "12_SP-UVR-3B-44100.pth", "13_SP-UVR-4B-44100-1.pth", "14_SP-UVR-4B-44100-2.pth", "15_SP-UVR-MID-44100-1.pth", "16_SP-UVR-MID-44100-2.pth", "17_HP-Wind_Inst-UVR.pth", "UVR-De-Echo-Aggressive.pth", "UVR-De-Echo-Normal.pth", "UVR-DeEcho-DeReverb.pth", "UVR-DeNoise-Lite.pth", "UVR-DeNoise.pth", "UVR-BVE-4B_SN-44100-1.pth", "MGM_HIGHEND_v4.pth", "MGM_LOWEND_A_v4.pth", "MGM_LOWEND_B_v4.pth", "MGM_MAIN_v4.pth"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Lower window size yield a higher output quality, but will take longer to process.
window_size = "320" #@param ["320", "512", "1024"]
#@markdown Intensity of primary stem extraction.
aggression = 5 #@param {type:"slider", min:0, max:100, step:1}
#@markdown Enable Test-Time-Augmentation; slow but improves quality.
tta = True #@param {type:"boolean"}
#@markdown Mirror the missing frequency range of the output.
high_end_process = False #@param {type:"boolean"}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(audio_input, output_folder, extensions, output_format, model, window_size, aggression, tta, high_end_process)

### ***Demucs***

In [ ]:
#@markdown #**Separation! Demucs Only**
import os
import glob
import subprocess
import yt_dlp

def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(f'{path}/temp', '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(audio_input, output_folder, extensions, output_format, model, shifts, overlap):
    found_files = []

    if checker(audio_input):
        downloader(audio_input)
        audio_input = f"{path}/temp"

    for audio_files in os.listdir(audio_input):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(audio_input, audio_files)
            if model == "drumsep":
              prompt = ["demucs", "--repo", f"{path}/models", f"--shifts={shifts}", f"--overlap={overlap}", "-o", f"{output_folder}", "-n", "drumsep", file_path]
            else:
              prompt = ["audio-separator", file_path, "-m", f"{model}", f"--output_dir={output_folder}", f"--output_format={output_format}", "--normalization=0.9", f"--demucs_shifts={shifts}", f"--demucs_overlap={overlap}"]
            print(f"Processing file: {audio_files}")
            subprocess.run(prompt)
            print(f"File: {audio_files} processed!")

    if audio_input == f"{path}/temp":
        temp_files = glob.glob(f"{path}/temp/*")
        for file in temp_files:
            os.remove(file)

#@markdown Input path for audio files or link:
audio_input = "Place the path or link here" #@param {type:"string"}
#@markdown You can paste the link to the video/audio from many sites, check the complete list [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)

#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "htdemucs_ft.yaml" #@param ["drumsep", "htdemucs_ft.yaml", "htdemucs.yaml", "hdemucs_mmi.yaml", "htdemucs_6s.yaml"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Number of predictions with random shifts, higher = slower but better quality.
shifts = 2 #@param {type:"slider", min:1, max:20, step:1}
#@markdown Amount of overlap between prediction windows.
overlap = 0.025 #@param {type:"slider", min:0.001, max:0.999, step:0.001}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(audio_input, output_folder, extensions, output_format, model, shifts, overlap)

# <small> **Extra**
Base code by [Blane187](https://github.com/Blane187)

In [ ]:
#@markdown #**Audio Downloader for Separation**
import os
import yt_dlp

#@markdown Link:
video_url = "https://youtu.be/L3P0LgwuPPo?si=8uQWCX9kiXgMxYdE"  #@param {type:"string"}
#@markdown You can paste the link to the video/audio from many sites, check the complete list [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)

#@markdown Audio name:
audio_name = "Harenchi"  #@param {type:"string"}
#@markdown Output path for downloaded audio file:
save_folder = "/content/drive/MyDrive/Separar"  #@param {type:"string"}
#@markdown Select the output format:
audio_format = "wav"  #@param ["wav", "flac", "mp3"]

def downloader(url, save_path, audio_format, audio_name):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': audio_format,
            'preferredquality': '192',
        }],
        'outtmpl': save_path,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

save_path = os.path.join(save_folder, audio_name)
downloader(video_url, save_path, audio_format, audio_name)
print("Download Complete!")